In [17]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [89]:
df_hedged_returns = pd.read_csv("data/q3/hedged_returns.csv",
                                   parse_dates=["date"],
                                   index_col="date")

df = df_hedged_returns.rename(columns={
    "australia_hedged_return_usd":   "AUS",
    "france_hedged_return_usd":      "FRA",
    "germany_hedged_return_usd":     "GER",
    "japan_hedged_return_usd":       "JPN",
    "switzerland_hedged_return_usd": "SWI",
    "uk_hedged_return_usd":          "UK"
})


tbill_df = pd.read_csv(
    "data/preprocess/tbills.csv",
    parse_dates=["date"],
    index_col="date"
)

tbill = tbill_df.rename(columns={"rf": "TBill1Mo"})["TBill1Mo"].iloc[1:-1]

#
#   div_df = pd.read_csv("data/q3/div_returns.csv", parse_dates=["date"], index_col="date")
#   div = div_df["DIV_return"]   # a pandas Series of DIV returns, index matches 2002-05-01 ... etc.
#

one_plus = 1.0 + df

### 4 Equity Index Momentum Strategy (MOM)

In [90]:
# Compute 11-month lookback cumulative return, then shift by 1 to impose 1-month lag:
cumprod_11 = one_plus.rolling(window=12).apply(np.prod, raw=True).shift(1) - 1

# Rank each row (month) ascending
ranks_11 = cumprod_11.rank(axis=1, method="first", ascending=True)

# Compute scaling factor Z so that ∑_{i: w>0} w = +1 and ∑_{i: w<0} w = -1.
N = df.shape[1]             
mean_rank = (N + 1) / 2.0    
if N % 2 == 0:
    Z = 8.0 / (N**2)       
else:
    Z = 8.0 / (N**2 - 1)   

# Compute MOM weights
w_mom = Z * (ranks_11 - mean_rank)

# Strategy return at month 
mom_ret = (w_mom * df).sum(axis=1, skipna=False)

# Decompose the “long leg” (w>0) and “short leg” (w<0) returns:
long_w_mom  = w_mom.clip(lower=0)
short_w_mom = w_mom.clip(upper=0)
mom_long_ret  = (long_w_mom  * df).sum(axis=1, skipna=False)
mom_short_ret = (short_w_mom * df).sum(axis=1, skipna=False)

# Align with T-Bill series, drop NaNs for clean statistics:
mom_df = pd.DataFrame({
    "MOM":   mom_ret,
    "LONG":  mom_long_ret,
    "SHORT": mom_short_ret,
    "TBill": tbill
}).dropna()

# Helper to compute annualized mean, vol, and Sharpe
def annualized_stats(returns, rf):
    """
    Given two pandas Series of monthly returns (decimal):
      - returns: strategy returns
      - rf:      risk-free returns
    Returns (annualized_mean, annualized_vol, annualized_sharpe).
    """
    excess = returns - rf
    m = returns.mean()
    sigma_m = returns.std()
    mean_ann = (1 + m)**12 - 1
    vol_ann = sigma_m * np.sqrt(12)
    sharpe = (excess.mean() / excess.std()) * np.sqrt(12)
    return mean_ann, vol_ann, sharpe

mean_mom, sigma_mom, sharpe_mom = annualized_stats(mom_df["MOM"], mom_df["TBill"])
mean_mL,  sigma_mL,  sharpe_mL  = annualized_stats(mom_df["LONG"], mom_df["TBill"])
mean_mS,  sigma_mS,  sharpe_mS  = annualized_stats(mom_df["SHORT"],mom_df["TBill"])

# One-sample t-test: H0: mean(MOM) = 0
t_stat_mom, p_val_mom = stats.ttest_1samp(mom_df["MOM"], popmean=0.0)

# Regress MOM on DIV (Newey-West errors, lag=1)
#reg_mom_div = pd.DataFrame({
#    "MOM": mom_ret,
#    "DIV": div
#}).dropna()

#X_m = sm.add_constant(reg_mom_div["DIV"])
#y_m = reg_mom_div["MOM"]
#mom_on_div = sm.OLS(y_m, X_m).fit(cov_type="HAC", cov_kwds={"maxlags": 1})

print("=== MOMENTUM (MOM) Strategy Statistics ===")
print(f"Overall MOM       : Annualized Return = {mean_mom:.2%}, "
      f"Volatility = {sigma_mom:.2%}, Sharpe = {sharpe_mom:.2f}")
print(f"MOM Long Leg      : Annualized Return = {mean_mL:.2%}, "
      f"Volatility = {sigma_mL:.2%}, Sharpe = {sharpe_mL:.2f}")
print(f"MOM Short Leg     : Annualized Return = {mean_mS:.2%}, "
      f"Volatility = {sigma_mS:.2%}, Sharpe = {sharpe_mS:.2f}")
print(f"MOM mean-return t-stat = {t_stat_mom:.3f}, p-value = {p_val_mom:.3f}\n")

#print(">>> Regression of MOM on DIV:")
#print(mom_on_div.summary())
#print("\n")


=== MOMENTUM (MOM) Strategy Statistics ===
Overall MOM       : Annualized Return = 32.27%, Volatility = 15.10%, Sharpe = 1.79
MOM Long Leg      : Annualized Return = 33.04%, Volatility = 17.84%, Sharpe = 1.55
MOM Short Leg     : Annualized Return = -0.59%, Volatility = 15.85%, Sharpe = -0.13
MOM mean-return t-stat = 8.703, p-value = 0.000



In [ ]:
# a)

In [3]:
# b)

In [4]:
# c)

### 5 Equity Index Long Term Reversal strategy (REV)

In [91]:
# Compute rolling 60-month product and rolling 12-month product:
rolling48 = one_plus.rolling(window=60).apply(lambda x: np.prod(x[:-11]), raw=False).shift(1)  - 1

# Rank each month ascending 
ranks_48 = rolling48.rank(axis=1, method="first", ascending=True)

# Compute scaling factor Z so that ∑_{i: w>0} w = +1 and ∑_{i: w<0} w = -1.
N = df.shape[1]             
mean_rank = (N + 1) / 2.0    
if N % 2 == 0:
    Z = 8.0 / (N**2)       
else:
    Z = 8.0 / (N**2 - 1) 
    
# Compute MOM weights
w_rev = Z * (mean_rank - ranks_48)

# Strategy return at t: sum_i [ w_rev[i,t] * df[i,t] ]
rev_ret = (w_rev * df).sum(axis=1, skipna=False)

# Decompose “long leg” and “short leg” for REV:
long_w_rev  = w_rev.clip(lower=0)
short_w_rev = w_rev.clip(upper=0)
rev_long_ret  = (long_w_rev  * df).sum(axis=1, skipna=False)
rev_short_ret = (short_w_rev * df).sum(axis=1, skipna=False)

# Align with T-Bill, drop NaNs:
rev_df = pd.DataFrame({
    "REV":   rev_ret,
    "LONG":  rev_long_ret,
    "SHORT": rev_short_ret,
    "TBill": tbill
}).dropna()

mean_rev, sigma_rev, sharpe_rev = annualized_stats(rev_df["REV"], rev_df["TBill"])
mean_rL,  sigma_rL,  sharpe_rL  = annualized_stats(rev_df["LONG"], rev_df["TBill"])
mean_rS,  sigma_rS,  sharpe_rS  = annualized_stats(rev_df["SHORT"],rev_df["TBill"])

# One-sample t-test for REV mean
t_stat_rev, p_val_rev = stats.ttest_1samp(rev_df["REV"], popmean=0.0)

# Regress REV on DIV (Newey-West lag=1)
#reg_rev_div = pd.DataFrame({
#    "REV": rev_ret,
#    "DIV": div
#}).dropna()

#X_r = sm.add_constant(reg_rev_div["DIV"])
#y_r = reg_rev_div["REV"]
#rev_on_div = sm.OLS(y_r, X_r).fit(cov_type="HAC", cov_kwds={"maxlags": 1})

print("=== REVERSAL (REV) Strategy Statistics ===")
print(f"Overall REV       : Annualized Return = {mean_rev:.2%}, "
      f"Volatility = {sigma_rev:.2%}, Sharpe = {sharpe_rev:.2f}")
print(f"REV Long Leg      : Annualized Return = {mean_rL:.2%}, "
      f"Volatility = {sigma_rL:.2%}, Sharpe = {sharpe_rL:.2f}")
print(f"REV Short Leg     : Annualized Return = {mean_rS:.2%}, "
      f"Volatility = {sigma_rS:.2%}, Sharpe = {sharpe_rS:.2f}")
print(f"REV mean-return t-stat = {t_stat_rev:.3f}, p-value = {p_val_rev:.3f}\n")

#print(">>> Regression of REV on DIV:")
#print(rev_on_div.summary())

=== REVERSAL (REV) Strategy Statistics ===
Overall REV       : Annualized Return = -12.91%, Volatility = 15.04%, Sharpe = -0.99
REV Long Leg      : Annualized Return = 2.93%, Volatility = 15.26%, Sharpe = 0.11
REV Short Leg     : Annualized Return = -15.42%, Volatility = 18.04%, Sharpe = -0.98
REV mean-return t-stat = -3.831, p-value = 0.000



In [83]:
import pandas as pd
import numpy as np

# Create a DataFrame with 100 months of fake returns (from 1 to 100)
dates = pd.date_range(start='2000-01-01', periods=200, freq='ME')
returns = pd.Series(np.arange(1, 201), index=dates)
df = pd.DataFrame({'US': returns})


# Apply rolling window of 60 months and compute product of t-60 to t-12 returns
# For each 60-month window, we remove the most recent 12 months → x[:-12]
lagged_cum_returns = df['US'].rolling(window=60).apply(lambda x: np.sum(x[:-11]), raw=False).shift(1)

print(lagged_cum_returns.iloc[60])

# Print the last value to verify
print("Lagged cumulative return at t=100 (i.e., product from month 40 to 88):")
print(lagged_cum_returns.iloc[-1])  # Should be product of values from 40 to 88

# Optional: verify manually
expected_product = np.sum(np.arange(1, 50))  # Inclusive of 40 to 88
print("Expected manual product from 40 to 88:")
print(expected_product)

1225.0
Lagged cumulative return at t=100 (i.e., product from month 40 to 88):
8036.0
Expected manual product from 40 to 88:
1225


In [5]:
# a)

In [6]:
# b) 

In [7]:
# c)

### Second version

In [ ]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats


# 2.1 Compute (1 + R) and shift by 1 so that at time t we use data through t-1:
returns_plus1 = df_ch + 1.0
shifted = returns_plus1.shift(1)

# 2.2 “12-month cumulative return from t-12 to t-1”: rolling window of length 12, then product − 1
mom_12m_cum = shifted.rolling(window=12, min_periods=12).apply(np.prod, raw=True) - 1.0

def compute_mom_portfolio(df_ch, mom_12m_cum):
    """
    Returns:
      • mom_weights: DataFrame (T × 6) of weights w_{i,t} at each t
      • mom_ret:     Series of MOM total returns R_t^{MOM}
      • mom_long:    Series of MOM long-leg returns
      • mom_short:   Series of MOM short-leg returns
    """
    N = df_ch.shape[1] 
    Z = 2.0 / (N * (N - 1))

    mom_weights = pd.DataFrame(index=df_ch.index, columns=df_ch.columns, dtype=float)

    for t in mom_12m_cum.index:
        if mom_12m_cum.loc[t].isna().any():
            continue
        # Rank by 12-month cum return at t (lowest=1, highest=6)
        ranks = mom_12m_cum.loc[t].rank(method="first")
        mid = (N + 1) / 2.0  # = 3.5
        w = Z * (ranks - mid)
        mom_weights.loc[t] = w.values

    # Portfolio return at t = sum_i w_{i,t} * R_{i,t}
    mom_ret = (mom_weights * df_ch).sum(axis=1)

    # Separate long and short legs
    mom_long = pd.Series(index=df_ch.index, dtype=float)
    mom_short = pd.Series(index=df_ch.index, dtype=float)

    for t in mom_ret.index:
        wt = mom_weights.loc[t]
        rt = df_ch.loc[t]
        if wt.isna().any():
            mom_long.loc[t] = np.nan
            mom_short.loc[t] = np.nan
            continue
        long_w = wt[wt > 0]
        short_w = wt[wt < 0]
        mom_long.loc[t] = (long_w * rt[long_w.index]).sum()
        mom_short.loc[t] = (short_w * rt[short_w.index]).sum()

    return mom_weights, mom_ret, mom_long, mom_short

mom_weights, mom_ret, mom_long, mom_short = compute_mom_portfolio(df_ch, mom_12m_cum)

mom_df = pd.DataFrame({
    "MOM": mom_ret,
    "MOM_long": mom_long,
    "MOM_short": mom_short
})

# -----------------------------------------------------------------------------------
# 2.3 Part 4(b): Compute Summary Statistics for MOM
# -----------------------------------------------------------------------------------
def summary_stats(series: pd.Series):
    """
    Given a pandas Series of monthly returns, compute:
      • Mean (monthly)
      • Std (monthly)
      • Annualized Sharpe = (mean/std)*sqrt(12)
      • t-stat for mean ≠ 0 and corresponding two-tailed p-value
      • Number of non-NA observations
    """
    r = series.dropna()
    T = len(r)
    mu = r.mean()
    sigma = r.std(ddof=1)
    sharpe = (mu / sigma) * np.sqrt(12) if sigma and T > 1 else np.nan
    t_stat = mu / (sigma / np.sqrt(T)) if sigma and T > 1 else np.nan
    pval = 2 * (1 - stats.t.cdf(abs(t_stat), df=T - 1)) if sigma and T > 1 else np.nan
    return {
        "Mean (monthly)": mu,
        "Std (monthly)": sigma,
        "Sharpe (ann.)": sharpe,
        "t-stat": t_stat,
        "p-value": pval,
        "Observations": T
    }

mom_stats = {leg: summary_stats(mom_df[leg]) for leg in ["MOM_long", "MOM_short", "MOM"]}

print("=== MOM Strategy Summary ===")
for leg, stats_dict in mom_stats.items():
    print(f"\n-- {leg} --")
    for k, v in stats_dict.items():
        print(f"{k}: {v:.6f}")

# -----------------------------------------------------------------------------------
# 2.4 Part 4(c): Regress MOM Returns on DIV Returns
# -----------------------------------------------------------------------------------
joined_mom = pd.concat([mom_df["MOM"], div_ret.rename("DIV")], axis=1).dropna()
Y_mom = joined_mom["MOM"]
X_mom = sm.add_constant(joined_mom["DIV"])
mom_model = sm.OLS(Y_mom, X_mom).fit()

print("\n=== Regression: MOM ~ DIV ===")
print(mom_model.summary())

# -----------------------------------------------------------------------------------
# 3.  Part 5(a): Construct the 5-Year (t-60 to t-12) Reversal Signal & Portfolio
# -----------------------------------------------------------------------------------
# Compute rolling 60-month and 12-month cumulative returns, shifted by 1 month
cum_60 = returns_plus1.rolling(window=60).apply(np.prod, raw=True)
cum_12 = returns_plus1.rolling(window=12).apply(np.prod, raw=True)

# Compute 5-year lagged (t-60 to t-12) cumulative return as (1+cum_60)/(1+cum_12)-1
rev_5yr_cum = (cum_60)/(cum_12) - 1.0

def compute_rev_portfolio(df_ch, rev_5yr_cum):
    """
    Returns:
      • rev_weights: DataFrame (T × 6) of weights w_{i,t} at each t
      • rev_ret:     Series of REV total returns R_t^{REV}
      • rev_long:    Series of REV long-leg returns
      • rev_short:   Series of REV short-leg returns
    """
    N = df_ch.shape[1]  # should be 6
    Z = 2.0 / (N * (N - 1))  # = 1/15 for N=6

    rev_weights = pd.DataFrame(index=df_ch.index, columns=df_ch.columns, dtype=float)

    for t in rev_5yr_cum.index:
        if rev_5yr_cum.loc[t].isna().any():
            continue
        # Rank by 5-year lagged cum return (t-60 to t-12) at date t
        ranks = rev_5yr_cum.loc[t].rank(method="first")
        mid = (N + 1) / 2.0  # = 3.5
        # w_{i,t} = Z * (mid - rank_i)
        w = Z * (mid - ranks)
        rev_weights.loc[t] = w.values

    rev_ret = (rev_weights * df_ch).sum(axis=1)

    rev_long = pd.Series(index=df_ch.index, dtype=float)
    rev_short = pd.Series(index=df_ch.index, dtype=float)
    for t in rev_ret.index:
        wt = rev_weights.loc[t]
        rt = df_ch.loc[t]
        if wt.isna().any():
            rev_long.loc[t] = np.nan
            rev_short.loc[t] = np.nan
            continue
        long_w = wt[wt > 0]
        short_w = wt[wt < 0]
        rev_long.loc[t] = (long_w * rt[long_w.index]).sum()
        rev_short.loc[t] = (short_w * rt[short_w.index]).sum()

    return rev_weights, rev_ret, rev_long, rev_short

rev_weights, rev_ret, rev_long, rev_short = compute_rev_portfolio(df_ch, rev_5yr_cum)

rev_df = pd.DataFrame({
    "REV": rev_ret,
    "REV_long": rev_long,
    "REV_short": rev_short
})

# -----------------------------------------------------------------------------------
# 3.2 Part 5(b): Compute Summary Statistics for REV
# -----------------------------------------------------------------------------------
rev_stats = {leg: summary_stats(rev_df[leg]) for leg in ["REV_long", "REV_short", "REV"]}

print("\n=== REV Strategy Summary ===")
for leg, stats_dict in rev_stats.items():
    print(f"\n-- {leg} --")
    for k, v in stats_dict.items():
        print(f"{k}: {v:.6f}")

# -----------------------------------------------------------------------------------
# 3.3 Part 5(c): Regress REV Returns on DIV Returns
# -----------------------------------------------------------------------------------
joined_rev = pd.concat([rev_df["REV"], div_ret.rename("DIV")], axis=1).dropna()
Y_rev = joined_rev["REV"]
X_rev = sm.add_constant(joined_rev["DIV"])
rev_model = sm.OLS(Y_rev, X_rev).fit()

print("\n=== Regression: REV ~ DIV ===")
print(rev_model.summary())

# -----------------------------------------------------------------------------------
# End of Script
# -----------------------------------------------------------------------------------
